In [14]:
import pandas as pd
import openai
import time

openai.api_key = 'put chatgpt api here'

df = pd.read_csv('apinegetive.csv')

def try_request(func, retries=5, delay=1, *args, **kwargs):
    for i in range(retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if isinstance(e, openai.error.ServiceUnavailableError):
                print("Service is currently unavailable, waiting for 60 seconds before retrying.")
                time.sleep(60)  # wait for 60 seconds before retrying
            elif '502' in str(e) and i < retries - 1:  # If a 502 status code was returned
                print(f"Encountered a 502 error, waiting for {delay} seconds before retrying.")
                time.sleep(delay)
            else:  # If the error is something else, re-raise it.
                raise

def get_sentiment(review_text):
    prompt = f"The following comment was left on the Amazon store:\n\"{review_text}\"\nBased on the content and tone of the comment, is it negative or positive?"
    message = [{"role": "system", "content": prompt}]
    response = try_request(openai.ChatCompletion.create, model="gpt-3.5-turbo", messages=message)
    sentiment = response['choices'][0]['message']['content']
    return sentiment.strip()

def get_emotion(review_text):
    prompt = f"""
    Please identify the dominant emotion expressed in the following review based on these definitions:

    
   -Sadness:The code sadness is assign to an end-user comment in the Amazon store where crowd-users express grief or pain
over a software feature, a bug or issue in the software application, or an issue reported in the software’s quality or with the overall
software. Generally, sadness occurs when a beloved one or an essential item is lost. Similarly, crowd-users express sadness
when unsatisfied with a single feature or overall software application in their reported comment in the Amazon store. Some
examples are For some time the app shows invalid file when trying to open almost any PDF, I used to love the app and reinstalled
hoping they fixed the problem, but it doesn’t work and Fun way to connect with my granddaughter. And I was unable to figure
out how to use. I am sad for it.
- Anger: The concept of anger is assign to end-user feedback in the Amazon store and refers to what stops crowd-users from
achieving the desired goals with either a single feature or overall software application. Also, a specific software feature or the
overall application behaves unexpectedly or unfairly. In general, anger is one of the seven universal emotions that arise when
we are not reaching our vision or are maltreated. At its most extreme, anger can be one of the most dangerous emotions due to its
possible link to violence. Such information is of pivotal importance for software and requirements engineers, as it might result
in the uninstallation of the software applications. Therefore, with the proposed approach, it can be identified earlier to earn user
satisfaction and improve the overall quality of the software application. Some examples of the anger concept in the Amazon
store are: I downloaded and deleted because of the difficult layout. Also, I got a problem like this: invalid file when trying to open. Can you please fix this? You’re bothering me, bro and They are big liars and stink. Every time I tried to open it, it
would crash and then tell me an unknown error had occurred.
-Disgust: The concept disgust is assign to end-user feedback in the Amazon store where crowd-users strongly dislike a
specific software feature, a bug or issue that has arrived in the software application, an issue reported in the software’s quality,
or the overall software application. In general, disgust is one of the seven universal emotions, defined by a strong dislike
for something unpleasant. In requirements engineering, it is essential to identify the disgust emotions of crowd-users in the
Amazon store, as they mainly describe the software application’s negative behavior, specifically the non-functional attributes.
For example, you can only make calls to users of this application. I couldn’t even sign up; it just exited out and said the app had
foreclosed three times, so I uninstalled it and got a better texting app that lets me create an account. And I’m on my Kindle Fire
HD. I don’t like this app. and I couldn’t even sign up; it just exited out and said the app forbade it three times, so I uninstalled
it and got a better texting app that lets me create an account. Don’t like it. Identifying such requirements-related information
earlier can help improve software applications’ quality and end-user satisfaction.
-Fear: The concept fear is assign to end-user feedback in the Amazon store where crowd-users sense or feel a possible threat
with a specific software feature or the overall software application. In general, fear is induced by the threat of harm, which can
be physiological, emotional, or mental in origin and can be real or imagined. In requirements and software engineering, fear
is often considered a negative emotion. Identifying it earlier in the software development phase can give the crowd-users
enough confidence in the software’s functionalities and overall performance. Some examples of fear emotions in the Amazon
store are useless. I was unable to make the simplest edit to my PDF. I wish I could get my money back, but Amazon makes
it impossible. and I installed this app, and that’s as far as I got. Every time I tried to open it, it would crash and then tell me
an unknown error had occurred. I uninstalled and reinstalled it a few times but never opened it. I can’t say if I liked it or not
because I never got to try it.
-Distrust: The label distrust is attributed to an end-user feedback in the Amazon store when users express apprehension regarding the reliability
or credibility of a software feature or the entire software application. Distrust can emerge when software consistently underperforms
or when there's perceived jeopardy to the user's safety, particularly in the realm of data privacy or potential misuse. For instance, 
a user expressed distrust by commenting, The new permissions really sour what is otherwise a great tool. Also, I am not a fan of apps 
that require the internet to run. I certainly don't need the net to be operational at my PC for 90%+ of my apps, why should I have the
net active on my phone? Such sentiments reflect concerns about unnecessary permissions and reliance on internet connectivity, contributing 
to an overall sense of distrust in the app. Identifying and addressing these expressions of distrust can be integral to enhancing user confidence 
and satisfaction.
-Frustration: The label frustration is assigned to end-user feedback in the Amazon store when users express dissatisfaction because software 
features do not operate as anticipated, there are recurring issues, or there's an absence of crucial functionality. For instance, one user expressed 
frustration by commenting, Continuously crashes. Cannot open even after restarts. No way to receive refund. Another user's review conveyed frustration with the words, 
Get an 'invalid file' error since Marshmallow. These comments exemplify experiences that hinder the user's goals, leading to a sense of frustration. Identifying and addressing these 
frustrations in a timely manner can significantly improve user satisfaction and the overall quality of the software application.
-Confusion: The code confusion is attributed to end-user feedback in the Amazon store when users express perplexity or lack of clarity regarding a software feature or the overall 
software application. This state of uncertainty is typically triggered by ambiguous instructions, poorly designed user interfaces, or complex functionalities that aren't intuitive.
A perfect exemplification of confusion can be seen in a user's comment, Same as below review. I have the same opinion as the person below me, Confused!!!!. This review illustrates 
a shared sentiment of bewilderment among users. Recognizing and resolving these moments of confusion can guide developers in refining the user interface and enhancing the overall user experience.
-Disappointment: The label disappointment is applied to end-user feedback in the Amazon store when users express a sentiment of letdown or dissatisfaction regarding a software feature or the overall
software application. Disappointment typically surfaces when the application fails to meet the users' expectations or when the promised benefits do not materialize in the user's experience. For example,
a user might express disappointment in a comment like, I’m not sure that it's working. Whenever I have it clean my computer, the result is 0 bytes removed and it continues to download. Not clear or comfortable with the results. JT. 
This review reflects a user's disappointment stemming from the perceived ineffectiveness of the software. Identifying and addressing these instances of disappointment can assist in making necessary adjustments to the software and help in managing user expectations better.
   
    Review: "{review_text}"
    """

    message = [{"role": "system", "content": prompt}]
    response = try_request(openai.ChatCompletion.create, model="gpt-3.5-turbo", messages=message)
    emotion = response['choices'][0]['message']['content']
    return emotion.strip()

# Process dataframe in chunks
chunk_size = 500  # Adjust based on your rate limit
chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]

for i, chunk in enumerate(chunks):
    for index, row in chunk.iterrows():
        sentiment = get_sentiment(row['Full_Review'])
        chunk.loc[index, 'sentiment'] = sentiment
        time.sleep(1)

    # Filter for negative reviews in the processed chunk
    chunk_negative = chunk[chunk['sentiment'].str.contains('negative')]

    # If sentiment is negative, further analyze for specific emotions
    for index, row in chunk_negative.iterrows():
        emotion = get_emotion(row['Full_Review'])
        chunk_negative.loc[index, 'emotion'] = emotion
        time.sleep(1)

    # Append the processed chunk to the output file
    if i == 0:  # For the first chunk, write the header
        chunk_negative.to_csv('resultofnegetive78000.csv', index=False, mode='a')
    else:  # For subsequent chunks, do not write the header
        chunk_negative.to_csv('resultofnegetive78000.csv', index=False, mode='a', header=False)
    
    print(f"Finished processing chunk {i + 1}/{len(chunks)}")
    time.sleep(60)  # Adjust based on your rate limit


/var/folders/y0/23chrxcn0d9dv4pxp6v708kr0000gn/T/ipykernel_45887/3877821903.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.loc[index, 'sentiment'] = sentiment


Encountered a 502 error, waiting for 1 seconds before retrying.
Encountered a 502 error, waiting for 1 seconds before retrying.
Encountered a 502 error, waiting for 1 seconds before retrying.


/var/folders/y0/23chrxcn0d9dv4pxp6v708kr0000gn/T/ipykernel_45887/3877821903.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_negative.loc[index, 'emotion'] = emotion


Finished processing chunk 1/5


/var/folders/y0/23chrxcn0d9dv4pxp6v708kr0000gn/T/ipykernel_45887/3877821903.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.loc[index, 'sentiment'] = sentiment
/var/folders/y0/23chrxcn0d9dv4pxp6v708kr0000gn/T/ipykernel_45887/3877821903.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_negative.loc[index, 'emotion'] = emotion


...

Encountered a 502 error, waiting for 1 seconds before retrying.
Finished processing chunk 3/5


/var/folders/y0/23chrxcn0d9dv4pxp6v708kr0000gn/T/ipykernel_45887/3877821903.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.loc[index, 'sentiment'] = sentiment
/var/folders/y0/23chrxcn0d9dv4pxp6v708kr0000gn/T/ipykernel_45887/3877821903.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_negative.loc[index, 'emotion'] = emotion


Finished processing chunk 4/5


/var/folders/y0/23chrxcn0d9dv4pxp6v708kr0000gn/T/ipykernel_45887/3877821903.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.loc[index, 'sentiment'] = sentiment


Finished processing chunk 5/5
